# SQL Avancé

Nous utiliserons le dialecte Transact-SQL (T-SQL) : [https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15](https://docs.microsoft.com/fr-fr/sql/t-sql/language-reference?view=sql-server-ver15)  

## Révisions

\- Schémas

\- Création de table

> \- Clé primaire
> 
> \- Clé étrangère
> 
> \- Unicité

## Création de base

In [ ]:
-- On supprime et on re-créé tout
IF (EXISTS (SELECT [name] FROM [master].[sys].[databases] WHERE [name] = 'Kiosque'))
    DROP DATABASE [Kiosque]

CREATE DATABASE [Kiosque]

GO

USE [Kiosque]

## Schémas

\- ref

\- data

Un schéma -\> regroupement d'éléments SQL (Tables, Vuews, ..) pour faciliter la gestion de permissions

In [ ]:
CREATE SCHEMA [data]

GO

CREATE SCHEMA [ref]

GO

## Création de Tables

In [ ]:
CREATE TABLE [data].[Utilisateur_UTL] (
    [Id] UNIQUEIDENTIFIER NOT NULL PRIMARY KEY DEFAULT NEWID(),
    [Login] NVARCHAR(50) NOT NULL UNIQUE,
    [Email] NVARCHAR(255) NOT NULL UNIQUE,
    [Actif] BIT NOT NULL DEFAULT 0
)

In [ ]:
CREATE TABLE [ref].[Role_ROL] (
    [Id] TINYINT NOT NULL PRIMARY KEY IDENTITY(1, 1),
    [Key] CHAR(10) NOT NULL,
    [Description] NVARCHAR(100) NOT NULL
)

In [ ]:
CREATE TABLE [data].[RoleUtilisateur_RUL] (
    [ROL_Id] TINYINT NOT NULL,
    [UTL_Id] UNIQUEIDENTIFIER NOT NULL,
    CONSTRAINT [pk_rol_utl] PRIMARY KEY ([ROL_Id], [UTL_Id]),
    CONSTRAINT [fk_RUL_ROL] FOREIGN KEY ([ROL_Id]) REFERENCES [ref].[Role_ROL]([Id]),
    CONSTRAINT [fk_RUL_UTL] FOREIGN KEY ([UTL_Id]) REFERENCES [data].[Utilisateur_UTL]([Id])
)

## Modification de tables

In [ ]:
ALTER TABLE [data].[Utilisateur_UTL]
    ADD [MotDePasse] NVARCHAR(512) NULL

In [ ]:
CREATE TABLE [ref].[Permission_PER] (
    [Id] INT IDENTITY(1,1) NOT NULL,
    [Nom] NVARCHAR(25) NOT NULL,
    [Description] NVARCHAR(256) NULL
)

In [ ]:
ALTER TABLE [ref].[Permission_PER]
    ADD CONSTRAINT [PK_PER] PRIMARY KEY ([Id])

## Opréations CRUD

Create, Read, Update, Delete

  

Create -\>  INSERT

Read -\> SELECT

Update -\> UPDATE

Delete -\> DELETE

In [ ]:
INSERT INTO [ref].[Role_ROL] ([Key], [Description])
VALUES
    ('admin', N'Administrateur'),
    ('guest', N'Visiteur'),
    ('market', N'Marketing')

In [ ]:
INSERT INTO [ref].[Role_ROL] ([Key], [Description])
VALUES
    ('guest', N'Autre visiteur')

In [ ]:
SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

In [ ]:
DELETE FROM [ref].[Role_ROL]
WHERE [Id] IN (
    SELECT [Id]FROM [ref].[Role_ROL]
    WHERE [Description] = N'Autre visiteur'
)

SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

In [ ]:
ALTER TABLE [ref].[Role_ROL]
    ADD [ROL_Key] AS LOWER([Key]),
    CONSTRAINT [uq_key] UNIQUE([ROL_Key])

In [ ]:
--INSERT INTO [ref].[Role_ROL] ([Key], [Description])
--VALUES
--    ('guest', N'Autre visiteur')

In [ ]:
UPDATE [ref].[Role_ROL]
SET [Description] = CONCAT(N'Utilisateur ', LOWER([Description]))

SELECT [Id], [Key], [Description] FROM [ref].[Role_ROL]

In [ ]:
INSERT INTO [data].[Utilisateur_UTL] ([Login], [Email], [Actif])
VALUES
    (N'alice', N'alice@u-picardie.fr', 1),
    (N'bob', N'bob@u-picardie.fr', 1),
    (N'carl', N'carl@u-picardie.fr', 0)

GO

DECLARE @UtlId UNIQUEIDENTIFIER
DECLARE @RolId TINYINT

SELECT TOP 1 @UtlId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'alice'

SELECT TOP 1 @RolId = [Id]
FROM [ref].[Role_ROL] (NOLOCK)
WHERE [ROL_Key] = 'admin'

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @UtlId AS [UTL_Id], @RolId AS [ROL_Id]


Pour la prochaine séance :

\- Ajouter le rôle Marketing à Alice

\- Ajouter le rôle Marketing à Bob

\- Ajouter le rôle Guest à Carl

In [ ]:

DECLARE @AliceId UNIQUEIDENTIFIER
DECLARE @BobId UNIQUEIDENTIFIER
DECLARE @CarlId UNIQUEIDENTIFIER
DECLARE @MarketId TINYINT
DECLARE @GuestId TINYINT

SELECT TOP 1 @AliceId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'alice'

SELECT TOP 1 @BobId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'bob'

SELECT TOP 1 @CarlId = [Id]
FROM [data].[Utilisateur_UTL] (NOLOCK)
WHERE [Login] = 'carl'

SELECT TOP 1 @MarketId = [Id]
FROM [ref].[Role_ROL] (NOLOCK)
WHERE [ROL_Key] = 'market'

SELECT TOP 1 @GuestId = [Id]
FROM [ref].[Role_ROL] (NOLOCK)
WHERE [ROL_Key] = 'guest'

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @AliceId AS [UTL_Id], @MarketId AS [ROL_Id]

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @BobId AS [UTL_Id], @MarketId AS [ROL_Id]

INSERT INTO [data].[RoleUtilisateur_RUL]([UTL_Id], [ROL_Id])
SELECT @CarlId AS [UTL_Id], @GuestId AS [ROL_Id]


Créer une table de liaison PermissionRole\_PRO entre Role et permission (schéma ref)

Ajouter trois permissions : 

\- CREATE\_CONTENT

\- READ\_CONTENT

\- EDIT\_CONTENT

Les liées au rôle admin

Lier uniquement READ\_CONTENT et EDIT\_CONTENT au rôle marketing

Lier READ\_CONTENT au rôle guest

In [ ]:
CREATE TABLE [ref].[PermissionRole_PRO] (
    [ROL_Id] TINYINT NOT NULL,
    [PER_Id] INT NOT NULL,
    CONSTRAINT [pk_rol_per] PRIMARY KEY ([ROL_Id], [PER_Id]),
    CONSTRAINT [fk_PRO_ROL] FOREIGN KEY ([ROL_Id]) REFERENCES [ref].[Role_ROL]([Id]),
    CONSTRAINT [fk_PRO_PER] FOREIGN KEY ([PER_Id]) REFERENCES [ref].[Permission_PER]([Id])
)

In [ ]:
INSERT INTO [ref].[Permission_PER] ([Nom], [Description])
VALUES
    (N'CREATE_CONTENT', N'Permission de créer du contenu'),
    (N'READ_CONTENT', N'Permission de consulter le contenu'),
    (N'EDIT_CONTENT', N'Permission d''éditer du contenu')

In [ ]:
DECLARE @AdminId TINYINT
DECLARE @MarketId TINYINT
DECLARE @GuestId TINYINT

SELECT TOP 1 @AdminId = [Id] FROM [ref].[Role_ROL] (NOLOCK) WHERE [ROL_Key] = 'admin'
SELECT TOP 1 @MarketId = [Id] FROM [ref].[Role_ROL] (NOLOCK) WHERE [ROL_Key] = 'market'
SELECT TOP 1 @GuestId = [Id] FROM [ref].[Role_ROL] (NOLOCK) WHERE [ROL_Key] = 'guest'

INSERT INTO [ref].[PermissionRole_PRO] ([ROL_Id], [PER_Id])
SELECT @AdminId, [Id] FROM [ref].[Permission_PER] (NOLOCK)
WHERE [Nom] IN (N'CREATE_CONTENT', N'READ_CONTENT', N'EDIT_CONTENT')

INSERT INTO [ref].[PermissionRole_PRO] ([ROL_Id], [PER_Id])
SELECT @MarketId, [Id] FROM [ref].[Permission_PER] (NOLOCK)
WHERE [Nom] IN (N'READ_CONTENT', N'EDIT_CONTENT')

INSERT INTO [ref].[PermissionRole_PRO] ([ROL_Id], [PER_Id])
SELECT @GuestId, [Id] FROM [ref].[Permission_PER] (NOLOCK)
WHERE [Nom] IN (N'READ_CONTENT')

![Jointures SQL](https://i.pinimg.com/originals/78/06/27/78062746cee62bdc112aad0582ae49c4.jpg)

Récupérer les utilsateurs avec la permission READ\_CONTENT

Puis Récupérer les utlisateurs avec la permission EDIT\_CONTENT

In [ ]:
SELECT DISTINCT [UTL].[Login]
    FROM [data].[Utilisateur_UTL] AS [UTL] (NOLOCK)

INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL] (NOLOCK)
    ON [RUL].[UTL_Id] = [UTL].[Id]

INNER JOIN [ref].[Role_ROL] AS [ROL] (NOLOCK)
    ON [ROL].[Id] = [RUL].[ROL_Id]

INNER JOIN [ref].[PermissionRole_PRO] AS [PRO] (NOLOCK)
    ON [PRO].[ROL_Id] = [ROL].[Id]

INNER JOIN [ref].[Permission_PER] AS [PER] (NOLOCK)
    ON [PER].[Id] = [PRO].[PER_Id]

WHERE [PER].[Nom] = N'READ_CONTENT'

In [ ]:
SELECT DISTINCT [UTL].[Login]
    FROM [data].[Utilisateur_UTL] AS [UTL] (NOLOCK)
INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL] (NOLOCK)
    ON [RUL].[UTL_Id] = [UTL].[Id]
INNER JOIN [ref].[Role_ROL] AS [ROL] (NOLOCK)
    ON [ROL].[Id] = [RUL].[ROL_Id]
INNER JOIN [ref].[PermissionRole_PRO] AS [PRO] (NOLOCK)
    ON [PRO].[ROL_Id] = [ROL].[Id]
INNER JOIN [ref].[Permission_PER] AS [PER] (NOLOCK)
    ON [PER].[Id] = [PRO].[PER_Id]
WHERE [PER].[Nom] = N'EDIT_CONTENT'

## Vues et Index

In [ ]:
CREATE OR ALTER VIEW [data].[view_PermissionUtilisateur]
AS
SELECT DISTINCT [UTL].[Id] AS [UTL_Id], [UTL].[Login]
              , [PER].[Id] AS [PER_Id], [PER].[Nom] AS [Permission]
    FROM [data].[Utilisateur_UTL] AS [UTL] (NOLOCK)

INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL] (NOLOCK)
    ON [RUL].[UTL_Id] = [UTL].[Id]

INNER JOIN [ref].[Role_ROL] AS [ROL] (NOLOCK)
    ON [ROL].[Id] = [RUL].[ROL_Id]

INNER JOIN [ref].[PermissionRole_PRO] AS [PRO] (NOLOCK)
    ON [PRO].[ROL_Id] = [ROL].[Id]

INNER JOIN [ref].[Permission_PER] AS [PER] (NOLOCK)
    ON [PER].[Id] = [PRO].[PER_Id]

WHERE [UTL].[Actif] = 1

In [ ]:
SELECT [UTL_Id], [Login], [PER_Id], [Permission] FROM [data].[view_PermissionUtilisateur]

In [ ]:
CREATE INDEX [IDX_UtlId_By_Login]
ON [data].[Utilisateur_UTL] ([Login], [Id])

In [ ]:
CREATE OR ALTER VIEW [data].[view_RoleUtilisateur]
WITH SCHEMABINDING
AS
SELECT [UTL].[Id] AS [UTL_Id], [UTL].[Login]
              , [ROL].[Id] AS [ROL_Id], [ROL].[Key] AS [Role]
    FROM [data].[Utilisateur_UTL] AS [UTL]

INNER JOIN [data].[RoleUtilisateur_RUL] AS [RUL]
    ON [RUL].[UTL_Id] = [UTL].[Id]

INNER JOIN [ref].[Role_ROL] AS [ROL]
    ON [ROL].[Id] = [RUL].[ROL_Id]

WHERE [UTL].[Actif] = 1

In [ ]:
CREATE UNIQUE CLUSTERED INDEX [IDX_RoleUtilisateur_By_UtlId]
ON [data].[view_RoleUtilisateur] ([UTL_Id], [Role])

In [ ]:
CREATE UNIQUE INDEX [IDX_RoleUtilisateur_By_UtlLogin]
ON [data].[view_RoleUtilisateur] ([Login], [Role])

# Fonctions

2 types :

\- Natives

\- UDF (User Defined Functions)

In [ ]:
SELECT TOP 1
    GETDATE() AS [Date],
    DATEADD(DAY, 2, GETDATE()) AS [Après demain],
    DATEDIFF(DAY, DATEADD(MONTH, 2, GETDATE()), GETDATE()) AS [Diff deux mois],
    CONCAT([Login], '@', 'u-picardie.fr') AS [Email],
    COALESCE(null, 'Test') AS [Coalesce]
FROM [data].[Utilisateur_UTL]

In [ ]:
CREATE FUNCTION [data].[fun_CountRolesForLogin](@Login NVARCHAR(50))
RETURNS INT
AS
BEGIN
    DECLARE @Retour INT

    SELECT @Retour = COUNT(*)
    FROM [data].[view_RoleUtilisateur]
    WHERE [Login] = @Login

    RETURN @Retour
END

In [ ]:
SELECT [data].[fun_CountRolesForLogin](N'alice')

In [ ]:
CREATE FUNCTION [data].[fun_GetRolesForLogin](@Login NVARCHAR(50))
RETURNS TABLE
AS
    RETURN SELECT [ROL_Id] AS [Id], [Role]
    FROM [data].[view_RoleUtilisateur]
    WHERE [Login]= @Login

In [ ]:
SELECT [Id], [Role] FROM [data].[fun_GetRolesForLogin](N'alice')

## Triggers et Curseurs

Un trigger (déclencheur) =\> un code éxécuté à un événement donné sur une table donnée

  

Curseur =\> Un espace mémoire qui va comporter un résultat de requête et nous permettre de le manipuler enregistrement par enregistrement

In [ ]:
ALTER TABLE [data].[Utilisateur_UTL]
    ADD [Edit] DATETIME NOT NULL DEFAULT GETDATE()

In [ ]:
CREATE OR ALTER TRIGGER [data].[trig_UTL_Edit]
    ON [data].[Utilisateur_UTL]
    AFTER UPDATE
AS
BEGIN
    SET NOCOUNT ON;

    DECLARE @UtlId UNIQUEIDENTIFIER
    DECLARE @NbItems INT

    SELECT @NbItems = COUNT(*) FROM INSERTED

    IF @NbItems > 1
    BEGIN
        DECLARE curs CURSOR FOR SELECT [Id] FROM INSERTED

        OPEN curs

        FETCH NEXT FROM curs INTO @UtlId
        WHILE (@@FETCH_STATUS = 0)
        BEGIN
            UPDATE [data].[Utilisateur_UTL]
            SET [Edit] = GETDATE()
            WHERE [Id] = @UtlId

            FETCH NEXT FROM curs INTO @UtlId
        END

        CLOSE curs
        DEALLOCATE curs
    END
    ELSE
    BEGIN
        SELECT TOP 1 @UtlId = [Id] FROM INSERTED

        UPDATE [data].[Utilisateur_UTL]
        SET [Edit] = GETDATE()
        WHERE [Id] = @UtlId
    END
END

Apparté : Itérateur

  

Itérateur : Structure qui va nous permettre de faire des boucles "à la main"

  

1. Déclarer/Récupérer l'itérateur
2. Demander le next/Demander si le next existe
3. Si next est définit, on le récupère
4. On fait notre traitement sur next
5. Retourner à l'étape 2, sinon quitter

## Transactions et procédures

In [ ]:
BEGIN TRANSACTION MA_TRANSACTION
BEGIN TRY
    INSERT INTO [ref].[Role_ROL] ([Key], [Description])
    VALUES (N'seo', N'Référencement')

    INSERT INTO [ref].[Role_ROL] ([Key], [Description])
    VALUES (N'admin', N'Administrateur')

    COMMIT TRANSACTION MA_TRANSACTION
END TRY
BEGIN CATCH
    ROLLBACK TRANSACTION MA_TRANSACTION

    SELECT ERROR_NUMBER(), ERROR_MESSAGE()
END CATCH

SELECT [Key], [Description]
FROM [ref].[Role_ROL]

In [ ]:
CREATE OR ALTER PROCEDURE [data].[prc_InsertUTL]
    @Login NVARCHAR(50),
    @IsActive BIT
AS
BEGIN
    SET NOCOUNT ON;

    BEGIN TRANSACTION INS_UTL
    BEGIN TRY
        DECLARE @Table TABLE (
            Id UNIQUEIDENTIFIER
        )
        DECLARE @RoleId TINYINT

        SELECT TOP 1 @RoleId = [Id]
        FROM [ref].[Role_ROL]
        WHERE [Key] = 'guest'

        INSERT INTO [data].[Utilisateur_UTL] ([Login], [Email], [Actif])
        OUTPUT INSERTED.Id
        INTO @Table
        VALUES (@Login, CONCAT(@Login, N'@u-picardie.fr'), @IsActive)

        INSERT INTO [data].[RoleUtilisateur_RUL]([ROL_Id], [UTL_Id])
        SELECT @RoleId, Id
        FROM @Table

        COMMIT TRANSACTION INS_UTL

        SELECT 0 AS [Number], N'' AS [Message]
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION INS_UTL

        SELECT ERROR_NUMBER() AS [Number], ERROR_MESSAGE() AS [Message]
    END CATCH
END

In [ ]:
EXEC [data].[prc_InsertUTL] @Login = N'didier', @IsActive = 1

## Usages avancées de T-SQL

Manipulation de XML

In [ ]:
DECLARE @XmlInfos AS XML = '<root>
    <editeur>
        <nom>Larousse</nom>
        <description>Editeur spécialisé en dictionnaires</description>
    </editeur>
    <editeur>
        <nom>Dupuis</nom>
        <description>Editeur de manuels scolaires</description>
    </editeur>
</root>'

DECLARE @Table TABLE (
    [Nom] NVARCHAR(MAX),
    [Description] NVARCHAR(MAX)
)

INSERT INTO @Table
SELECT 'Nom' = x.r.value('nom[1]', 'nvarchar(max)'),
       'Description' = x.r.value('description[1]', 'nvarchar(max)')
FROM @XmlInfos.nodes('/root/editeur') x(r)

SELECT * FROM @Table

Récupérer le dernier ID inséré

In [ ]:
CREATE TABLE [data].[Temp_TMP] (
    [Id] BIGINT IDENTITY(1,1) NOT NULL PRIMARY KEY,
    [Nom] NVARCHAR(50) NOT NULL
)

GO

INSERT INTO [data].[Temp_TMP]([Nom])
VALUES (N'Alice'), (N'Bob'), (N'Carl')

SELECT SCOPE_IDENTITY()

Conversion de données

In [ ]:
SELECT CAST(N'12' AS INT)

## CTE - Common Table Expression

In [ ]:
-- Id Magazine|Quantité;Id Magazine|Quantité.....
DECLARE @InfosNumeros NVARCHAR(MAX) = N'1;5|2;3|3;9'

;WITH CTEInfosMag AS (
    SELECT CAST(PARSENAME(REPLACE([value], ';', '.'), 2) AS BIGINT) AS [Id],
           CAST(PARSENAME(REPLACE([value], ';', '.'), 1) AS SMALLINT) AS [Quantite]
    FROM STRING_SPLIT(@InfosNumeros, '|')
)

SELECT * FROM CTEInfosMag

GO

-- Erreur, changement de portée, ne peut plus être consommée
SELECT * FROM CTEInfosMag


# Projet

- Groupes de 1 à 5
- Rendu
    - Dictionnaire de données (justifier vos architecturaux)
    - 1 fichier de création de la BDD (SQL ou Notebook)
    - 1 fichier de remplissage de votre BDD (SQL ou Notebook)
    - 1 schéma graphique de votre BDD
- Logiciel de gestion de magazines :
    - Permissions utilisateurs
    - Magazines (1 éditeur, N distributeurs)
    - Editeurs (N magazines)
    - Distributeurs (N magazines)
    - Formats (papier A4, Papier autre, numérique) -\> 1 par magazine
- Ne pas en faire plus que demandé

## Contraintes

- 1 vue doit sortir les noms de magazines par éditeurs
- Les différentes tables doivent avoir des index pertinents (ex: Deux magazines ne peuvent pas avoir le même nom car ce sont des marques déposées)
- Une fonction doit retourner l'ensemble des distributeurs et éditeur d'un magazine
- On veut que chaque table métier comporte une date de création et une date de modification de la donnée qui soient maintenus à jour.
- Lors de la création d'un utilisateur, lui assigner systématiquement la ou les permissions les plus basses
- Créer une procédure permettant d'insérer un magazine (nouveau ou existant) auprès d'un éditeur (si il n'en a pas déjà un) et d'un distributeur (sans doublons). En cas d'erreur, on revient à l'état initial.